In [1]:
# 构建计算图——LSTM模型
#    embedding
#    LSTM
#    fc
#    train_op
# 训练流程代码
# 数据集封装
#    api: next_batch(batch_size)
# 词表封装:
#    api: sentence2id(text_sentence): 句子转换id
# 类别的封装：
#    api: category2id(text_category).

#import tensorflow as tf

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import os
import sys
import numpy as np
import math

#tf.logging.set_verbosity(tf.logging.INFO)  2.0格式以上变化了
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
# 词向量大小
num_embedding_size = 16,
# 步长
num_timesteps = 50,
# lstm单元输出维度（又叫输出神经元数）
num_lstm_nodes = [32, 32],
# 层数
num_lstm_layers = 2,
# 全连接输出维度，最后一维
num_fc_nodes = 32,
batch_size = 100,
# 控制梯度，梯度上线
clip_lstm_grads = 1.0,
# 学习率
learning_rate = 0.001,
# 词频最低下限
num_word_threshold = 10,
    



train_file = 'D:/BaiduNetdiskDownload/Rnn图像语义全部资料/深度学习之神经网络（CNN RNN GAN）算法原理+实战/课程数据/text_classification_data/cnews.train.seg.txt'
val_file = 'D:/BaiduNetdiskDownload/Rnn图像语义全部资料/深度学习之神经网络（CNN RNN GAN）算法原理+实战/课程数据/text_classification_data/cnews.val.seg.txt'
test_file = 'D:/BaiduNetdiskDownload/Rnn图像语义全部资料/深度学习之神经网络（CNN RNN GAN）算法原理+实战/课程数据/text_classification_data/cnews.test.seg.txt'
vocab_file = 'D:/BaiduNetdiskDownload/Rnn图像语义全部资料/深度学习之神经网络（CNN RNN GAN）算法原理+实战/课程数据/text_classification_data/cnews.vocab.txt'
category_file = 'D:/BaiduNetdiskDownload/Rnn图像语义全部资料/深度学习之神经网络（CNN RNN GAN）算法原理+实战/课程数据/text_classification_data/cnews.category.txt'
output_folder = 'D:/BaiduNetdiskDownload/Rnn图像语义全部资料/深度学习之神经网络（CNN RNN GAN）算法原理+实战/课程数据/text_classification_data/run_text_rnn'

content_num_file='D:/BaiduNetdiskDownload/Rnn图像语义全部资料/深度学习之神经网络（CNN RNN GAN）算法原理+实战/课程数据/text_classification_data/content_num.txt'


if not os.path.exists(output_folder):
    os.mkdir(output_folder)

#### 下面的意思是词表和类别的封装

In [3]:
class Vocab:
    def __init__(self, filename, num_word_threshold):
        #词典
        self._word_to_id = {}
        # <UNK> 的id（初始值）
        self._unk = -1
         # 频率下限
        self._num_word_threshold = num_word_threshold
        # 将词典读出来存到dict里
        self._read_dict(filename)
    
    def _read_dict(self, filename):
        with open(filename, 'r',encoding='utf-8') as f:
            lines = f.readlines()
        for line in lines:
            word, frequency = line.strip('\r\n').split('\t')
            word = word
            frequency = int(frequency)
#             print(type(frequency)) #int
#             print(type(self._num_word_threshold))#tuple  python的成员变量是元组，int不能和元组直接比较
#             print(self._num_word_threshold[0])#所以要直接去取出来，不然总是报错
            # 低于下限不要
            if frequency < self._num_word_threshold[0]:
                continue
            idx = len(self._word_to_id)
            if word == '<UNK>':
                 # 刷新UNK的id
                self._unk = idx
            self._word_to_id[word] = idx
            #print(self._unk) 0,这玩意是为了给下面做填充用的 ，padding
    
    # 如果没有word返回UNK
    def word_to_id(self, word):
        return self._word_to_id.get(word, self._unk)
    
    @property
    def unk(self):
        return self._unk
    
    def size(self):
        # 返回大小
        return len(self._word_to_id)
    
    def sentence_to_id(self, sentence):
        # 分词进字典里吧id取出来变成list
        word_ids = [self.word_to_id(cur_word) \
                    for cur_word in sentence.split()]
        return word_ids


class CategoryDict:
    def __init__(self, filename):
        self._category_to_id = {}
        # 读取类别并存入字典，给每个一个id,这里一直有个错误就是读取的文件要是utf-8的，之前一直报错
        with open(filename, 'r',encoding='utf-8') as f:
            lines = f.readlines()
        for line in lines:
            category = line.strip('\r\n')
            idx = len(self._category_to_id)
            self._category_to_id[category] = idx
    
    def size(self):
        return len(self._category_to_id)
        
    def category_to_id(self, category):
        if not category in self._category_to_id:
            raise Execption(
                "%s is not in our category list" % category_name)
        return self._category_to_id[category]
        
vocab = Vocab(vocab_file, num_word_threshold)
vocab_size = vocab.size()
#tf.compat.v1.logging.info('vocab_size: %d' % vocab_size)
print('vocab_size: %d' % vocab_size)
test_str='的 在 了 是'
print (vocab.sentence_to_id(test_str))


category_vocab = CategoryDict(category_file)
num_classes = category_vocab.size()
#tf.compat.v1.logging.info('num_classes: %d' % num_classes)
print('num_classes: %d' % num_classes)
test_str = '时尚'
# tf.compat.v1.logging.info(
#     'label: %s, id: %d' % (
#         test_str,
#         category_vocab.category_to_id(test_str)))

print('label:%s, id: %d' % (
        test_str,
        category_vocab.category_to_id(test_str)))

vocab_size: 77331
[2, 4, 6, 7]
num_classes: 10
label:时尚, id: 5


### 下面的是数据集的封装

In [4]:
class TextDataSet:
    def __init__(self, filename, vocab, category_vocab, num_timesteps):
        self._vocab = vocab
        self._category_vocab = category_vocab
        self._num_timesteps = num_timesteps
        # matrix矩阵
        self._inputs = []
        # vector 列表
        self._outputs = []
        self._indicator = 0
        #解析文件，就是把inputs和outputs填充的过程
        self._parse_file(filename)
    
    def _parse_file(self, filename):
        #提示我们要加载了
        print('Loading data from %s', filename)
        chen=[]
        with open(filename, 'r',encoding='utf-8') as f:
            lines = f.readlines()
        for line in lines:
            label, content = line.strip('\r\n').split('\t')
            #把lable转成id
            id_label = self._category_vocab.category_to_id(label)
            #print(content)为什么这种语句都是把所有的东西全部输出了
            #把内容转成id
            id_words = self._vocab.sentence_to_id(content)
            #下面的地址是：D:\BaiduNetdiskDownload\Rnn图像语义全部资料\深度学习之神经网络（CNN RNN GAN）算法原理+实战
            #\资料\coding-259-master\coding-259\py2\07-rnn-text-classification  我也不知道为什么会放到了代码那里去
            with open('content_num_file','w')as f:
                 for i in range (len(id_words)):
                        f.write(str(id_words[i]))
                        #f.write("成功了")
                        f.write('\r')       
            #chen.append(id_words)
            #print(type(id_words)) list
            #print(type(self._num_timesteps)) 依然是老错误，这东西是元组 tuples 所以从下一句开始出错了
            id_words = id_words[0: self._num_timesteps[0]]#拿到前50个值，下标从0开始 
            #print(id_words)
            #padding 填充,这玩意百分之九十多都是0，给进去的句子是以换行为一句的，要是他没有50个词就补呗
            padding_num = self._num_timesteps[0] - len(id_words)
            #print(padding_num)
            #差多少就补多少0，unk是_vocab的一个成员函数，所以可以直接调用
            id_words = id_words + [
                self._vocab.unk for i in range(padding_num)]
            self._inputs.append(id_words)
            self._outputs.append(id_label)
        #print(self._inputs[1]) 
        #print(self._outputs)不能这么搞，会全部弄出来的。现在的input和output都是全部的资料，
        
        #将输入数据（列表的列表，元组的元组，元组的列表等）转换为矩阵形式
        self._inputs = np.asarray(self._inputs, dtype = np.int32)
        self._outputs = np.asarray(self._outputs, dtype = np.int32)
        #随机化
        self._random_shuffle()
    
    def _random_shuffle(self):
        p = np.random.permutation(len(self._inputs))
       # print(p) 生成了len(self._inputs)那么长的随机数据,然后赋值给intput和outputs,
       #说不清，看下一个cell，我做了一个实验，output是lable值，应该只是把随机种子赋值过来，值还是自己的值
        self._inputs = self._inputs[p]
        self._outputs = self._outputs[p]
    
    #获得下一个minbatch
    def next_batch(self, batch_size):
        # 当获取的指针超过数据集大小时，归0
        end_indicator = self._indicator + batch_size
        if end_indicator > len(self._inputs):
            self._random_shuffle()
            self._indicator = 0
            end_indicator = batch_size
        # 归0还超过就报错
        if end_indicator > len(self._inputs):
            raise Execption("batch_size: %d is too large" % batch_size)
        
        #取值
        batch_inputs = self._inputs[self._indicator: end_indicator]
        batch_outputs = self._outputs[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_inputs, batch_outputs

    
    #分别把train val test的数据传进去
train_dataset = TextDataSet(
    train_file, vocab, category_vocab, num_timesteps) 
val_dataset = TextDataSet(
    val_file, vocab, category_vocab, num_timesteps)
test_dataset = TextDataSet(
    test_file, vocab, category_vocab, num_timesteps)

print (train_dataset.next_batch(2))
print (val_dataset.next_batch(2))
print (test_dataset.next_batch(1))  




Loading data from %s D:/BaiduNetdiskDownload/Rnn图像语义全部资料/深度学习之神经网络（CNN RNN GAN）算法原理+实战/课程数据/text_classification_data/cnews.train.seg.txt
Loading data from %s D:/BaiduNetdiskDownload/Rnn图像语义全部资料/深度学习之神经网络（CNN RNN GAN）算法原理+实战/课程数据/text_classification_data/cnews.val.seg.txt
Loading data from %s D:/BaiduNetdiskDownload/Rnn图像语义全部资料/深度学习之神经网络（CNN RNN GAN）算法原理+实战/课程数据/text_classification_data/cnews.test.seg.txt
(array([[ 1173,   501,  3181,     0,  6906, 16789,    11,   145, 28500,
         9681,   156,   599,   456,    82,    78,    68,    23,  1166,
           44,   269,     1,   160,  5734,   649,  1480,   270,     1,
        33181,     2,   501,   941, 28455,     1,    58,    63,  3181,
          204, 63944, 52552,  6906, 15017,     1,    35,     7,   145,
        13608,  1978,  2188,    29,     2],
       [ 1891,     2, 49112,  1274,  8144,     6, 12688,    10,  7245,
           17,   250,    16,  1891,     2, 49112,  1274,     1,  8144,
            6, 12688,    10,  7245,     3, 30316, 

In [100]:
chen= np.random.permutation(5)
print(chen)

chen_inputs=[6,7,8,9,10]
#print(type(chen_inputs))#list

chen_inputs=np.asarray(chen_inputs, dtype = np.int32)
print(chen_inputs) #[6,7,8,9,10]
#print(type(chen_inputs)) #矩阵

chen_inputs=chen_inputs[chen]
print(chen_inputs)

[2 4 0 1 3]
[ 6  7  8  9 10]
<class 'numpy.ndarray'>
[ 8 10  6  7  9]


In [7]:
hao=8
print(hao)
print(str(hao))


8
8


### enbedding与one-hot编码 很不错的讲解

(46条消息) embedding_lookup的学习笔记_FBeetle的博客-CSDN博客
https://blog.csdn.net/hit0803107/article/details/98377030

In [19]:

#不要删除，这个是下面的测试
input_ids = tf.placeholder(dtype=tf.int32, shape=[None])
_input_ids = tf.placeholder(dtype=tf.int32, shape=[3, 2])


embedding_param = tf.Variable(np.identity(8, dtype=np.int32))   # 生成一个8x8的单位矩阵
input_embedding = tf.nn.embedding_lookup(embedding_param, input_ids)
_input_embedding = tf.nn.embedding_lookup(embedding_param, _input_ids)

sess = tf.InteractiveSession()
## 含有tf.Variable的环境下，因为tf中建立的变量是没有初始化的，也就是在debug时还不是一个tensor量，
    #而是一个Variable变量类型
sess.run(tf.global_variables_initializer())


print('embedding:')
print(embedding_param.eval())

var1 = [1, 2, 6, 4, 2, 5, 7]
print('\n var1:')
print(var1)

print('\nprojecting result:')
print(sess.run(input_embedding, feed_dict={input_ids: var1}))

var2 = [[1, 4], [6, 3], [2, 5]]
print('\n _var2:')
print(var2)

print('\n _projecting result:')
print(sess.run(_input_embedding, feed_dict={_input_ids: var2}))


embedding:
[[1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1]]

 var1:
[1, 2, 6, 4, 2, 5, 7]

projecting result:
[[0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 1 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1]]

 _var2:
[[1, 4], [6, 3], [2, 5]]

 _projecting result:
[[[0 1 0 0 0 0 0 0]
  [0 0 0 0 1 0 0 0]]

 [[0 0 0 0 0 0 1 0]
  [0 0 0 1 0 0 0 0]]

 [[0 0 1 0 0 0 0 0]
  [0 0 0 0 0 1 0 0]]]


In [27]:
chen1= 1.0 / math.sqrt(8 / 3.0)
print(chen1)

0.6123724356957945


In [8]:
#通过这个可以看出来tf.variable的东西是三个变量的，
x=tf.Variable(0.)
y=tf.Variable(1.)
print(x)
print(y)

<tf.Variable 'Variable:0' shape=() dtype=float32_ref>
<tf.Variable 'Variable_1:0' shape=() dtype=float32_ref>


In [ ]:
下面到处是这种 with tf.variable_scope 

In [5]:
def create_model( vocab_size, num_classes):
    #步长
    #num_timesteps = num_timesteps
    #batch_size = batch_size
    
    #占位符 （三个参数分别是类型，形状（batch_size行，num_timesteps列，没有就是不固定），名字）
    inputs = tf.placeholder(tf.int32, (batch_size, num_timesteps))
    outputs = tf.placeholder(tf.int32, (batch_size, ))
    
    
#     Dropout就是在不同的训练过程中随机扔掉部分神经元。也就是让某个神经元的激活值以一定的概率p，
#     让其停止工作，这次训练过程中不更新权值，也不参加神经网络的计算。
#     tf.nn.dropout(x, keep_prob, noise_shape=None, seed=None,name=None)
#     第一个参数x：指输入第二个参数keep_prob: 设置神经元被选中的概率,在初始化时keep_prob是一个占位符,
#     keep_prob = tf.placeholder(tf.float32) 。
#     tensorflow在run时设置keep_prob具体的值，例如keep_prob: 0.5

    
   
    keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')
    
    # 保存模型运行进度
    global_step = tf.Variable(
        tf.zeros([], tf.int64), name = 'global_step', trainable=False)
    
     # embedding的值初始化到-1到1
    embedding_initializer = tf.random_uniform_initializer(-1.0, 1.0)
    
    #写成下面这样会好理解一些 scope理解成作用域，就是范围，与get_variable配合使用
    with tf.variable_scope('embedding', initializer = embedding_initializer):
        
        #把上面的[-1，1]的embeddings弄成一个vocab_size长，num_embedding_size宽的，重新赋给自己
        embeddings = tf.get_variable( 'embedding',[vocab_size, num_embedding_size],tf.float32)
        
        # [1, 10, 7] -> [embeddings[1], embeddings[10], embeddings[7]] 
        #将一个个词转化成embedding，看上面的enbedding与one-hot编码，你也别管怎么转了
        embed_inputs = tf.nn.embedding_lookup(embeddings, inputs)
        
     # 初始化随机值,先别管是什么，反正就是一个数字，看上面的测试
    scale = 1.0 / math.sqrt(num_embedding_size + num_lstm_nodes[-1]) / 3.0
    print("num_embedding_size:"+num_embedding_size+"/r")
    print("num_lstm_nodes[-1]:"+num_lstm_nodes[-1]+"/r")
    print(scale)
    
    
    lstm_init = tf.random_uniform_initializer(-scale, scale)
    with tf.variable_scope('lstm_nn', initializer = lstm_init):
        # 双层所以写一个list
        cells = []
        for i in range(num_lstm_layers):
            #basic+lstm+cell tf中的api直接实现了，num_lstm_nodes[i]是输入的大小
            #用于存储LSTM单元的state_size,zero_state和output state的元组。按顺序存储两个元素(c,h),
            #其中c是隐藏状态，h是输出。只有在state_is_tuple=True是才使用。
            cell = tf.contrib.rnn.BasicLSTMCell(num_lstm_nodes[i],state_is_tuple = True)
            #把上一层的cell作为这一层的输入，进行dropout
            cell = tf.contrib.rnn.DropoutWrapper(cell,output_keep_prob = keep_prob)
            cells.append(cell)
        #把两个cell拼接到一起
        cell = tf.contrib.rnn.MultiRNNCell(cells)
        
        # 初始化隐含状态为0
        initial_state = cell.zero_state(batch_size, tf.float32)
        
        #是tensorflow封装的用来实现递归神经网络（RNN）的函数
        # rnn_outputs: [batch_size, num_timesteps, lstm_outputs[-1]]
        rnn_outputs, _ = tf.nn.dynamic_rnn(
            cell, embed_inputs, initial_state = initial_state)
        #取最后一层，细化的东西再说吧
        # [batch_size, 1, lstm_outputs[-1]（32）]
        last = rnn_outputs[:, -1, :]
    
    #类似于均匀分布的东西
    fc_init = tf.uniform_unit_scaling_initializer(factor=1.0)
    
    # 输出和全连接层拼接
    with tf.variable_scope('fc', initializer = fc_init):
        # 上面lstm的cell的输出拿过来
        # 激活函数用relu
        # 第二个参数是输出维度大小，就是最后一个参数,这里是没有变化的
        fc1 = tf.layers.dense(last, 
                              num_fc_nodes,
                              activation = tf.nn.relu,
                              name = 'fc1')
        # dropout
        fc1_dropout = tf.contrib.layers.dropout(fc1, keep_prob)
        
        # 在经过一层全连接映射到多少num_classes个类别上 
        logits = tf.layers.dense(fc1_dropout,
                                 num_classes,
                                 name = 'fc2')
    
    
    with tf.name_scope('metrics'):
        #交叉熵损失函数，类别与标签的交叉熵
        softmax_loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits, labels = outputs)
        # 计算均值
        loss = tf.reduce_mean(softmax_loss)
        # [0, 1, 5, 4, 2] -> argmax: 2
        # 这里第一个参数就是softmax，第二个参数意思是取第几维最大，比如是1的话就是在第一维里去最大
        y_pred = tf.argmax(tf.nn.softmax(logits),
                           1, 
                           output_type = tf.int32)
        
        # 计算准确率，看看算对多少个
        correct_pred = tf.equal(outputs, y_pred)
        
        # 求均值，tf.cast  将数据转换成 tf.float32 类型
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
    #定义op(operation) 而且是train的op
    with tf.name_scope('train_op'):
        # 获得所有可训练的变量（优化器优化列表中的变量）
        tvars = tf.trainable_variables()
        
        # 看看这些变量叫啥
        for var in tvars:
            tf.logging.info('variable name: %s' % (var.name))
            
        # tf.gradients求导（梯度下降，对每个训练变量求偏导，每个方向都往下走）
        # 不超过hps.clip_lstm_grads，防止梯度爆炸
        grads, _ = tf.clip_by_global_norm(tf.gradients(loss, tvars), clip_lstm_grads)
        
        # Adam优化器
        optimizer = tf.train.AdamOptimizer(learning_rate)
        
        # 这里是应用，将梯度grads作用到变量上（看这个名字）,让函数吧global_step回调
        train_op = optimizer.apply_gradients(
            zip(grads, tvars), global_step = global_step)#zip将变量和梯度绑定在一起，先记住着吧
    
    return ((inputs, outputs, keep_prob),
            (loss, accuracy),
            (train_op, global_step))

placeholders, metrics, others = create_model(vocab_size, num_classes)

inputs, outputs, keep_prob = placeholders
loss, accuracy = metrics
train_op, global_step = others



TypeError: Error converting shape to a TensorShape: Dimension value must be integer or None or have an __index__ method, got value '(100,)' with type '<class 'tuple'>'.

In [7]:
init_op = tf.global_variables_initializer()
#keep_prob的 drop的概率
train_keep_prob_value = 0.8
test_keep_prob_value = 1.0

#跑多少次
#num_train_steps = 10000
num_train_steps = 100

# Train: 99.7%
# Valid: 92.7%
# Test:  93.2%
with tf.Session() as sess:
    sess.run(init_op)
    for i in range(num_train_steps):
        batch_inputs, batch_labels = train_dataset.next_batch(
            batch_size)
        # 那三个占位符输进去
        # 计算loss, accuracy, train_op, global_step的图
        outputs_val = sess.run([loss, accuracy, train_op, global_step],
                               feed_dict = {
                                   inputs: batch_inputs,
                                   outputs: batch_labels,
                                   keep_prob: train_keep_prob_value,
                               })
        loss_val, accuracy_val, _, global_step_val = outputs_val
         # 两百个batch输出一次
        if global_step_val % 20 == 0:
            print("Step: %5d, loss: %3.3f, accuracy: %3.3f"
                            % (global_step_val, loss_val, accuracy_val))
            
            
        # 一千个batch校验一次
        if global_step_val % 1000 == 0:
            accuracy_eval = eval_holdout(sess, accuracy, val_dataset, hps.batch_size)
            accuracy_test = eval_holdout(sess, accuracy, test_dataset, hps.batch_size)
            print("Step: %5d, val_accuracy: %3.3f, test_accuracy: %3.3f"
                            % (global_step_val, accuracy_eval, accuracy_test))



TypeError: unsupported operand type(s) for +: 'int' and 'tuple'